### Import mix module 

In [1]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
import argparse

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.9985486850848575

In [5]:
def haaqi_reward_fn(solution: np.ndarray) -> float:
    thre = solution[0]
    thre = thre - 15
    ratio = solution[1]
    ratio = ratio + 5
    attack = solution[2]
    attack = attack + 10
    release = solution[3] 
    release = release + 300
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return 1-score

In [7]:
###inital
haaqi_reward_fn(np.array([1, 0, 1, -50]))

0.09726680463629833

## Ranking

### GA

In [15]:
###Worst result for GA Until the 11.July
###haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))
####Worst result for GA Until the 12.July 
haaqi_reward_fn(np.array([-14.9, 3.3, -9.0, -193]))

0.25147015798225136

### RL in Continous

In [24]:
####Worst result for GA Until the 12.July [-1.2019364  -5.0801964  -3.631278  -54.986313]
haaqi_reward_fn(np.array([-1.2019,-5.0801,-3.6313,-54.9863]))

0.3335415127030843

### Import Discrete RL module

In [ ]:
import Optimiser

In [ ]:
import Optimiser.discrete_RL_train
from Optimiser.discrete_RL_train import discrete_RL_train as RLTrain

In [ ]:
trainner = RLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [26]:
###Worst result for discrete ReinforcementL Until the 11.July
haaqi_reward_fn(np.array([ -3.9 ,  -0.5 ,  -4.4 , -52.72]))

After LUFS, the mixing ouput in the RMS, Total: -9.24dB, Clipping Ratio&Cliped Num: (0.0011848958333333334, 455)
Current degraded thre:-18.9, ratio: 3.5, attack:5.699999999999999,release:247.38000000000002 bring the HAAQI score to 0.8496384255903743


0.15036157440962572

In [4]:
Noise_Generator_MP3.EraseTheMp3Mixing()

### Import Continous RL

In [11]:
import Optimiser

In [12]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

2025-07-12 16:27:25.699189: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-12 16:27:25.906668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-12 16:27:25.906721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-12 16:27:25.939702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-12 16:27:26.012197: I tensorflow/core/platform/cpu_feature_guar

In [13]:
trainner = CRLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.
[step   0] mean entropy per-dim = 5.5707
final reward before udpate: -1000000000.0
final reward after udpate: 0.13959818
updated final_solution= [ -5.7188005  -3.241129   -2.7341576 -47.553318 ]
train_step no.= 1
best_solution of this generation= [ -5.7188005  -3.241129   -2.7341576 -47.553318 ]
best step reward= 0.14
avg step reward= 0.084
act_std: tf.Tensor([0.9740769 0.9740769 0.9740769 0.9740769], shape=(4,), dtype=float32)
act_mean: tf.Tensor([-0.06840445 -0.02574186  0.0351551   0.00981691], shape=(4,), dtype=float32)
best_step_index: [1, 9]
 
[step   1] mean entropy per-dim = 5.5707
final reward before udpate: 0.13959818
final reward after udpate: 0.1457996
updated final_solution= [ -2.957088    5.2322173  -3.5029109 -50.32147  ]
[step   2] mean entropy per-dim = 5.5702
[step   3] mean entropy per-dim = 5.5703
[step   4] mean entropy per-dim = 5.5702
final reward before udpate: 0.1457996
fi

([0.13959818,
  0.1457996,
  0.1457996,
  0.1457996,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.16578723,
  0.17183438,
  0.17183438,
  0.25615206,
  0.25615206,
  0.32864282,
  0.32864282,
  0.32864282,
  0.32864282,
  0.32864282,
  0.32864282],
 array([ -1.2019364,  -5.0801964,  -3.631278 , -54.986313 ], dtype=float32),
 0.32864282)

### Import GA module

In [6]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [7]:
GA_Opt = GeneticOptimiser()

In [8]:
GA_Opt.ga_init_env()

In [9]:
GA_Opt.set_fitnessfun(haaqi_reward_fn)

In [10]:
GA_Opt.run()

sol_per_pop: 30, num_genes: 4, num_generations: 30, preset_population: [[1.0, 0.0, 1.0, -50.0]]


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:753: UserWarning: The percentage of genes to mutate (mutation_percent_genes=0.2) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/pygad/pygad.py:1149: UserWarning: Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number of genes is large.
  warnings.warn("Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of genera

Generation :  1
The last best Solution :  {(-14.5, 0.4, -8.2, -158)}
Fitness of the last best solution : {0.17863494786431056}
Generation :  2
The last best Solution :  {(-14.5, 3.6, -8.2, -152)}
Fitness of the last best solution : {0.21636713041053268}
Generation :  3
The last best Solution :  {(-14.5, 3.6, -8.2, -152)}
Fitness of the last best solution : {0.22850532848238503}
Generation :  4
The last best Solution :  {(-14.5, 3.4, -8.2, -158)}
Fitness of the last best solution : {0.22850532848238503}
Generation :  5
The last best Solution :  {(-14.5, 3.6, -9.0, -158)}
Fitness of the last best solution : {0.23060909500263138}
Generation :  6
The last best Solution :  {(-14.5, 2.6, -9.0, -193)}
Fitness of the last best solution : {0.24070948118443136}
Generation :  7
The last best Solution :  {(-14.5, 2.6, -9.0, -193)}
Fitness of the last best solution : {0.24338912132691082}
Generation :  8
The last best Solution :  {(-14.5, 2.6, -9.0, -193)}
Fitness of the last best solution : {0.243